In [11]:
import sys, subprocess, time, warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import rbf_kernel

from qiskit.circuit.library import ZZFeatureMap, TwoLocal, ZFeatureMap
from qiskit_aer.primitives import Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms.classifiers import PegasosQSVC
from qiskit_machine_learning.optimizers import COBYLA
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.utils import algorithm_globals
from qiskit.primitives import Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute

In [2]:
# 1. Wczytanie plików i przygotowanie etykiet
counts = pd.read_csv("countsAll_fixed_07_07_23.csv", delimiter="\t").T
metadata = pd.read_csv("SampleInfo_fixed_08_07_23.csv", delimiter=";").set_index("id")
metadata["label"] = metadata["GroupAlternative"].apply(lambda x: 0 if x == "Asymptomatic controls" else 1)
metadata = metadata[metadata["RealLocation"] != "Institute 5"]

In [3]:
# 2. Scalanie
df = counts.merge(metadata, left_index=True, right_index=True)
X = df.drop(columns=metadata.columns)
y = df["label"]

In [4]:
# 3. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X.values, y.values, train_size=0.7, random_state=42, shuffle=True
)

In [5]:
# 3. PCA redukcja do kilku qubitów (np. 4)
pca = PCA(n_components=2, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [6]:
# 4. Skalowanie danych do przedziału [0, π]
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
# 5. Konfiguracja parametrów PegasosQSVC
num_qubits = X_train_scaled.shape[1]
tau = 100
C = 1000

algorithm_globals.random_seed = 12345
feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=1)
sampler = Sampler()
fidelity = ComputeUncompute(sampler=sampler)
qkernel = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)

pegasos = PegasosQSVC(quantum_kernel=qkernel, C=C, num_steps=tau)

In [8]:
# 6. Trening i ewaluacja
t0 = time.time()
pegasos.fit(X_train_scaled, y_train)
train_time = time.time() - t0

score_train = pegasos.score(X_train_scaled, y_train)
score_test = pegasos.score(X_test_scaled, y_test)

print(f"Training time: {train_time:.2f} s")
print(f"Train accuracy: {score_train:.3f}")
print(f"Test accuracy: {score_test:.3f}")

AlgorithmError: 'Sampler job failed!'

In [9]:
X_train_sub = X_train_scaled[:200]
y_train_sub = y_train[:200]

In [10]:
sampler = Sampler(options={"shots": 256})

In [12]:
K_train = rbf_kernel(X_train_scaled)
K_test = rbf_kernel(X_test_scaled, X_train_scaled)

pegasos = PegasosQSVC(precomputed=True, C=1000, num_steps=100)
pegasos.fit(K_train, y_train)
print("Train acc:", pegasos.score(K_train, y_train))
print("Test acc:", pegasos.score(K_test, y_test))

Train acc: 0.8252427184466019
Test acc: 0.8349514563106796
